## Por Ciclovía

In [ ]:
!pip install geopandas
!pip install shapely

In [1]:
#rutas

ruta_cv = r"C:\Users\UTIC\Desktop\CV_2024\CV_2024.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\5. EMCA\Puntos AFO\puntos aforo.gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\5. EMCA\Crecimiento CICLV\CV_AfoTot_20_23.gpkg"
ruta_gpkg_salida_excel = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\5. EMCA\Crecimiento CICLV\CV_AfoTot_20_23.xlsx"

In [2]:
import geopandas as gpd

# Cargar el archivo shapefile de ciclovías
ciclovias = gpd.read_file(ruta_cv)

# Realizar dissolve por el campo CICLV
ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
ciclovias_buffer = ciclovias_dissolved.copy()
ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# Crear un GeoDataFrame a partir del buffer
ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

# Cargar las capas de puntos de aforo
puntos_aforo_20 = gpd.read_file(ruta_pun_afo, layer="pun_afo_20")
puntos_aforo_21 = gpd.read_file(ruta_pun_afo, layer="pun_afo_21")
puntos_aforo_22 = gpd.read_file(ruta_pun_afo, layer="pun_afo_22")
puntos_aforo_23 = gpd.read_file(ruta_pun_afo, layer="pun_afo_23")

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    aforo_total = []
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        total_aforo = aforo_intersect["Aforo"].sum()
        total_puntos = len(aforo_intersect)
        promedio_aforo = total_aforo / total_puntos if total_puntos > 0 else 0
        
        aforo_total.append(total_aforo)
        puntos_total.append(total_puntos)
        ciclovias_buffer_gdf.loc[idx, f"AFO_{year_suffix}"] = total_aforo
        ciclovias_buffer_gdf.loc[idx, f"PUN_{year_suffix}"] = total_puntos
        ciclovias_buffer_gdf.loc[idx, f"PROM_{year_suffix}"] = promedio_aforo

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_20, "20")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_21, "21")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_22, "22")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_23, "23")


In [3]:
import numpy as np

# Función para calcular la tasa de cambio ciclista
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["PROM_20"]
        prom_21 = row["PROM_21"]
        prom_22 = row["PROM_22"]
        prom_23 = row["PROM_23"]
        
        # Filtrar los años con información válida
        values = [(20, prom_20), (21, prom_21), (22, prom_22), (23, prom_23)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio entre años consecutivos
        tasas_cambio = []
        for i in range(len(values) - 1):
            año_inicial, valor_inicial = values[i]
            año_final, valor_final = values[i + 1]
            tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
            tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio)
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
ciclovias_buffer_gdf = calcular_tasa_cambio(ciclovias_buffer_gdf)


In [4]:
# Función para calcular el promedio ajustado de ciclistas por punto
def calcular_promedio_ajustado(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["PROM_20"]
        prom_21 = row["PROM_21"]
        prom_22 = row["PROM_22"]
        prom_23 = row["PROM_23"]
        
        # Filtrar los años con información válida
        values = [prom_20, prom_21, prom_22, prom_23]
        values = [v for v in values if v != 0]
        
        if len(values) == 0:
            # Si no hay años con datos, establecer el promedio ajustado como NaN
            ciclovias_gdf.loc[idx, "PROM_AJUSTADO"] = np.nan
        else:
            # Calcular el promedio ajustado
            promedio_ajustado = np.mean(values)
            ciclovias_gdf.loc[idx, "PROM_AJUSTADO"] = promedio_ajustado

    return ciclovias_gdf

# Aplicar la función de cálculo de promedio ajustado
ciclovias_buffer_gdf = calcular_promedio_ajustado(ciclovias_buffer_gdf)

In [6]:
# Guardar el resultado en un archivo geopackage
ciclovias_buffer_gdf.to_file(ruta_gpkg_salida, layer="CICLV_AFO_20_23", driver="GPKG")

# Guardar el resultado en un archivo excel
ciclovias_buffer_gdf.to_excel(ruta_gpkg_salida_excel)

## Por Nodo ciclista

In [31]:
#rutas

ruta_nodos = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\5. EMCA\Nodos\Nodos_20_23_v2.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\5. EMCA\Puntos AFO\puntos aforo.gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\5. EMCA\Nodos\Nodos_AfoTot_20_23.gpkg"

In [32]:
import geopandas as gpd

# Cargar el archivo shapefile de ciclovías
nodos = gpd.read_file(ruta_nodos)

# # Realizar dissolve por el campo CICLV
# ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# # Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
# ciclovias_buffer = ciclovias_dissolved.copy()
# ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# # Crear un GeoDataFrame a partir del buffer
# ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

# Cargar las capas de puntos de aforo
puntos_aforo_20 = gpd.read_file(ruta_pun_afo, layer="pun_afo_20")
puntos_aforo_21 = gpd.read_file(ruta_pun_afo, layer="pun_afo_21")
puntos_aforo_22 = gpd.read_file(ruta_pun_afo, layer="pun_afo_22")
puntos_aforo_23 = gpd.read_file(ruta_pun_afo, layer="pun_afo_23")

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    aforo_total = []
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        total_aforo = aforo_intersect["Aforo"].sum()
        total_puntos = len(aforo_intersect)
        promedio_aforo = total_aforo / total_puntos if total_puntos > 0 else 0
        
        aforo_total.append(total_aforo)
        puntos_total.append(total_puntos)
        ciclovias_buffer_gdf.loc[idx, f"AFO_{year_suffix}"] = total_aforo
        ciclovias_buffer_gdf.loc[idx, f"PUN_{year_suffix}"] = total_puntos
        ciclovias_buffer_gdf.loc[idx, f"PROM_{year_suffix}"] = promedio_aforo

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
nodos = procesar_aforo(nodos, puntos_aforo_20, "20")
nodos = procesar_aforo(nodos, puntos_aforo_21, "21")
nodos = procesar_aforo(nodos, puntos_aforo_22, "22")
nodos = procesar_aforo(nodos, puntos_aforo_23, "23")


In [34]:
import numpy as np

# Función para calcular la tasa de cambio ciclista
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["PROM_20"]
        prom_21 = row["PROM_21"]
        prom_22 = row["PROM_22"]
        prom_23 = row["PROM_23"]
        
        # Filtrar los años con información válida
        values = [(20, prom_20), (21, prom_21), (22, prom_22), (23, prom_23)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio entre años consecutivos
        tasas_cambio = []
        for i in range(len(values) - 1):
            año_inicial, valor_inicial = values[i]
            año_final, valor_final = values[i + 1]
            tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
            tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio)
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
nodos = calcular_tasa_cambio(nodos)


In [35]:
# Función para calcular el promedio ajustado de ciclistas por punto
def calcular_promedio_ajustado(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["PROM_20"]
        prom_21 = row["PROM_21"]
        prom_22 = row["PROM_22"]
        prom_23 = row["PROM_23"]
        
        # Filtrar los años con información válida
        values = [prom_20, prom_21, prom_22, prom_23]
        values = [v for v in values if v != 0]
        
        if len(values) == 0:
            # Si no hay años con datos, establecer el promedio ajustado como NaN
            ciclovias_gdf.loc[idx, "PROM_AJUSTADO"] = np.nan
        else:
            # Calcular el promedio ajustado
            promedio_ajustado = np.mean(values)
            ciclovias_gdf.loc[idx, "PROM_AJUSTADO"] = promedio_ajustado

    return ciclovias_gdf

# Aplicar la función de cálculo de promedio ajustado
nodos = calcular_promedio_ajustado(nodos)

In [36]:
#recalcular fid

nodos['fid']= nodos.index

In [37]:
# Guardar el resultado en un archivo geopackage
nodos.to_file(ruta_gpkg_salida, layer="NODOS_AFO_20_23", driver="GPKG")

## Por Ciclovía (mibici)

In [1]:
#rutas

ruta_cv = r"C:\Users\UTIC\Desktop\CV_2024\CV_2024.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\Pun Afo (MiBici)\Puntos aforo (MiBici).gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\Pun Afo (MiBici)\CV_AfoTot_20_23.gpkg"

In [3]:
import geopandas as gpd

# Cargar el archivo shapefile de ciclovías
ciclovias = gpd.read_file(ruta_cv)

# Realizar dissolve por el campo CICLV
ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
ciclovias_buffer = ciclovias_dissolved.copy()
ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# Crear un GeoDataFrame a partir del buffer
ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

# Cargar las capas de puntos de aforo
puntos_aforo_20 = gpd.read_file(ruta_pun_afo, layer="AFOROS_2020")
puntos_aforo_21 = gpd.read_file(ruta_pun_afo, layer="AFOROS_2021")
puntos_aforo_22 = gpd.read_file(ruta_pun_afo, layer="AFOROS_2022")
puntos_aforo_23 = gpd.read_file(ruta_pun_afo, layer="AFOROS_2023")

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    aforo_total = []
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        total_aforo = aforo_intersect["Conteo"].sum()
        total_puntos = len(aforo_intersect)
        promedio_aforo = total_aforo / total_puntos if total_puntos > 0 else 0
        
        aforo_total.append(total_aforo)
        puntos_total.append(total_puntos)
        ciclovias_buffer_gdf.loc[idx, f"AFO_{year_suffix}"] = total_aforo
        ciclovias_buffer_gdf.loc[idx, f"PUN_{year_suffix}"] = total_puntos
        ciclovias_buffer_gdf.loc[idx, f"PROM_{year_suffix}"] = promedio_aforo

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_20, "20")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_21, "21")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_22, "22")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_23, "23")


In [4]:
import numpy as np

# Función para calcular la tasa de cambio ciclista
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["PROM_20"]
        prom_21 = row["PROM_21"]
        prom_22 = row["PROM_22"]
        prom_23 = row["PROM_23"]
        
        # Filtrar los años con información válida
        values = [(20, prom_20), (21, prom_21), (22, prom_22), (23, prom_23)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio entre años consecutivos
        tasas_cambio = []
        for i in range(len(values) - 1):
            año_inicial, valor_inicial = values[i]
            año_final, valor_final = values[i + 1]
            tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
            tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio)
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
ciclovias_buffer_gdf = calcular_tasa_cambio(ciclovias_buffer_gdf)


In [5]:
# Función para calcular el promedio ajustado de ciclistas por punto
def calcular_promedio_ajustado(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["PROM_20"]
        prom_21 = row["PROM_21"]
        prom_22 = row["PROM_22"]
        prom_23 = row["PROM_23"]
        
        # Filtrar los años con información válida
        values = [prom_20, prom_21, prom_22, prom_23]
        values = [v for v in values if v != 0]
        
        if len(values) == 0:
            # Si no hay años con datos, establecer el promedio ajustado como NaN
            ciclovias_gdf.loc[idx, "PROM_AJUSTADO"] = np.nan
        else:
            # Calcular el promedio ajustado
            promedio_ajustado = np.mean(values)
            ciclovias_gdf.loc[idx, "PROM_AJUSTADO"] = promedio_ajustado

    return ciclovias_gdf

# Aplicar la función de cálculo de promedio ajustado
ciclovias_buffer_gdf = calcular_promedio_ajustado(ciclovias_buffer_gdf)

In [6]:
# Guardar el resultado en un archivo geopackage
ciclovias_buffer_gdf.to_file(ruta_gpkg_salida, layer="CICLV_AFO_20_23", driver="GPKG")

## Por ciclovía (delitos)

In [ ]:
#filtrar por año

In [27]:
#rutas

ruta_cv = r"C:\Users\UTIC\Desktop\CV_2024\CV_2024.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\4. RED CV\DELITOS\delitos totales amg.gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\4. RED CV\DELITOS\CV_DelTot_20_23.gpkg"

In [ ]:
import geopandas as gpd
import pandas as pd

# Cargar el archivo shapefile de ciclovías
ciclovias = gpd.read_file(ruta_cv)

# Realizar dissolve por el campo CICLV
ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
ciclovias_buffer = ciclovias_dissolved.copy()
ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# Crear un GeoDataFrame a partir del buffer
ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

#cargar delitos
puntos_aforo = gpd.read_file(ruta_pun_afo, layer="delitos totales amg")

# Primero, convierte la columna de fechas a formato datetime
puntos_aforo['fecha'] = pd.to_datetime(puntos_aforo['fecha'], format='%d/%m/%Y')

# Filtrar los registros para cada año
puntos_aforo_19 = puntos_aforo[puntos_aforo['fecha'].dt.year == 2019]
puntos_aforo_20 = puntos_aforo[puntos_aforo['fecha'].dt.year == 2020]
puntos_aforo_21 = puntos_aforo[puntos_aforo['fecha'].dt.year == 2021]
puntos_aforo_22 = puntos_aforo[puntos_aforo['fecha'].dt.year == 2022]
puntos_aforo_23 = puntos_aforo[puntos_aforo['fecha'].dt.year == 2023]

# Guardar el resultado en un archivo geopackage
puntos_aforo_19.to_file(ruta_pun_afo, layer="DEL_19", driver="GPKG")
puntos_aforo_20.to_file(ruta_pun_afo, layer="DEL_20", driver="GPKG")
puntos_aforo_21.to_file(ruta_pun_afo, layer="DEL_21", driver="GPKG")
puntos_aforo_22.to_file(ruta_pun_afo, layer="DEL_22", driver="GPKG")
puntos_aforo_23.to_file(ruta_pun_afo, layer="DEL_23", driver="GPKG")

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        
        # Filtra los puntos que intersectan con la geometría del buffer
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        # Cuenta el número de puntos en la intersección
        total_puntos = len(aforo_intersect)
        
        # Agrega el conteo al GeoDataFrame
        puntos_total.append(total_puntos)
        ciclovias_buffer_gdf.loc[idx, f"DEL_{year_suffix}"] = total_puntos

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_19, "19")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_20, "20")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_21, "21")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_22, "22")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_23, "23")

In [33]:
import numpy as np

# Función para calcular la tasa de cambio de delitos
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de DEL para cada año
        del_19 = row["DEL_19"]
        del_20 = row["DEL_20"]
        del_21 = row["DEL_21"]
        del_22 = row["DEL_22"]
        del_23 = row["DEL_23"]
        
        # Obtener el año de construcción de la ciclovía y convertirlo a entero
        ano_const = int(row["ANO_CONST"])
        
        # Filtrar los años con información válida
        values = [(19, del_19), (20, del_20), (21, del_21), (22, del_22), (23, del_23)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio basadas en el año de construcción
        tasas_cambio = []
        if ano_const in [2020, 2021, 2022, 2023]:
            # Caso A: Comparar el año previo a la construcción, el año de construcción y el año posterior
            relevant_years = [(ano_const - 1) % 100, ano_const % 100, (ano_const + 1) % 100]
            relevant_values = [v for v in values if v[0] in relevant_years]
            
            print(f"Index: {idx}, Año Construcción: {ano_const}, Relevant Years: {relevant_years}, Relevant Values: {relevant_values}")
            
            if len(relevant_values) < 2:
                ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
                continue
            
            for i in range(len(relevant_values) - 1):
                año_inicial, valor_inicial = relevant_values[i]
                año_final, valor_final = relevant_values[i + 1]
                tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
                tasas_cambio.append(tasa_cambio)
                
        else:
            # Caso B: Comparar todos los años disponibles
            for i in range(len(values) - 1):
                año_inicial, valor_inicial = values[i]
                año_final, valor_final = values[i + 1]
                tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
                tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio) if tasas_cambio else np.nan
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
ciclovias_buffer_gdf = calcular_tasa_cambio(ciclovias_buffer_gdf)

Index: 16SP, Año Construcción: 2023, Relevant Years: [22, 23, 24], Relevant Values: [(22, 74.0), (23, 63.0)]
Index: AORT, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 55.0), (21, 45.0), (22, 48.0)]
Index: AUNI, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 1.0), (22, 1.0)]
Index: BOSQ, Año Construcción: 2022, Relevant Years: [21, 22, 23], Relevant Values: [(21, 3.0), (23, 3.0)]
Index: CHAP, Año Construcción: 2023, Relevant Years: [22, 23, 24], Relevant Values: [(22, 11.0), (23, 10.0)]
Index: CJVC, Año Construcción: 2022, Relevant Years: [21, 22, 23], Relevant Values: [(21, 5.0), (22, 8.0), (23, 5.0)]
Index: EQLE, Año Construcción: 2023, Relevant Years: [22, 23, 24], Relevant Values: [(22, 6.0), (23, 2.0)]
Index: GDPE, Año Construcción: 2020, Relevant Years: [19, 20, 21], Relevant Values: [(19, 353.0), (20, 216.0), (21, 146.0)]
Index: GIGA, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 186.0), 

In [34]:
# Guardar el resultado en un archivo geopackage
ciclovias_buffer_gdf.to_file(ruta_gpkg_salida, layer="CICLV_DEL_20_23", driver="GPKG")

## Por Ciclovía (accidentes)

In [28]:
#rutas

ruta_cv = r"C:\Users\UTIC\Desktop\CV_2024\CV_2024.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\4. RED CV\ACCIDENTES\accidentes_19_22.gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\4. RED CV\ACCIDENTES\CV_AccTot_20_23.gpkg"

In [29]:
import geopandas as gpd

# Cargar el archivo shapefile de ciclovías
ciclovias = gpd.read_file(ruta_cv)

# Realizar dissolve por el campo CICLV
ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
ciclovias_buffer = ciclovias_dissolved.copy()
ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# Crear un GeoDataFrame a partir del buffer
ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

# Cargar las capas de puntos de aforo
puntos_aforo_19 = gpd.read_file(ruta_pun_afo, layer="ACCIDENTES_2019")
puntos_aforo_20 = gpd.read_file(ruta_pun_afo, layer="ACCIDENTES_2020")
puntos_aforo_21 = gpd.read_file(ruta_pun_afo, layer="ACCIDENTES_2021")
puntos_aforo_22 = gpd.read_file(ruta_pun_afo, layer="ACCIDENTES_2022")

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    aforo_total = []
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        total_aforo = aforo_intersect["NUMPOINTS"].sum()
        total_puntos = len(aforo_intersect)
        promedio_aforo = total_aforo / total_puntos if total_puntos > 0 else 0
        
        aforo_total.append(total_aforo)
        puntos_total.append(total_puntos)
        ciclovias_buffer_gdf.loc[idx, f"ACC_{year_suffix}"] = total_aforo
        # ciclovias_buffer_gdf.loc[idx, f"PUN_{year_suffix}"] = total_puntos
        # ciclovias_buffer_gdf.loc[idx, f"PROM_{year_suffix}"] = promedio_aforo

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_19, "19")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_20, "20")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_21, "21")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_22, "22")


In [30]:
import numpy as np

# Función para calcular la tasa de cambio de delitos
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de DEL para cada año
        del_19 = row["ACC_19"]
        del_20 = row["ACC_20"]
        del_21 = row["ACC_21"]
        del_22 = row["ACC_22"]
        
        # Obtener el año de construcción de la ciclovía y convertirlo a entero
        ano_const = int(row["ANO_CONST"])
        
        # Filtrar los años con información válida
        values = [(19, del_19), (20, del_20), (21, del_21), (22, del_22)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio basadas en el año de construcción
        tasas_cambio = []
        if ano_const in [2020, 2021, 2022, 2023]:
            # Caso A: Comparar el año previo a la construcción, el año de construcción y el año posterior
            relevant_years = [(ano_const - 1) % 100, ano_const % 100, (ano_const + 1) % 100]
            relevant_values = [v for v in values if v[0] in relevant_years]
            
            print(f"Index: {idx}, Año Construcción: {ano_const}, Relevant Years: {relevant_years}, Relevant Values: {relevant_values}")
            
            if len(relevant_values) < 2:
                ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
                continue
            
            for i in range(len(relevant_values) - 1):
                año_inicial, valor_inicial = relevant_values[i]
                año_final, valor_final = relevant_values[i + 1]
                tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
                tasas_cambio.append(tasa_cambio)
                
        else:
            # Caso B: Comparar todos los años disponibles
            for i in range(len(values) - 1):
                año_inicial, valor_inicial = values[i]
                año_final, valor_final = values[i + 1]
                tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
                tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio) if tasas_cambio else np.nan
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
ciclovias_buffer_gdf = calcular_tasa_cambio(ciclovias_buffer_gdf)

Index: 16SP, Año Construcción: 2023, Relevant Years: [22, 23, 24], Relevant Values: [(22, 26.0)]
Index: AORT, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 4.0), (21, 7.0), (22, 8.0)]
Index: AUNI, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 1.0), (21, 3.0), (22, 1.0)]
Index: BOSQ, Año Construcción: 2022, Relevant Years: [21, 22, 23], Relevant Values: [(22, 2.0)]
Index: CJVC, Año Construcción: 2022, Relevant Years: [21, 22, 23], Relevant Values: [(21, 2.0)]
Index: EQLE, Año Construcción: 2023, Relevant Years: [22, 23, 24], Relevant Values: [(22, 2.0)]
Index: GDPE, Año Construcción: 2020, Relevant Years: [19, 20, 21], Relevant Values: [(19, 52.0), (20, 18.0), (21, 19.0)]
Index: GIGA, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 26.0), (21, 16.0), (22, 16.0)]
Index: GILP, Año Construcción: 2021, Relevant Years: [20, 21, 22], Relevant Values: [(20, 31.0), (21, 7.0), (22, 18.0)]
Index: HDGO, Año 

In [31]:
# Guardar el resultado en un archivo geopackage
ciclovias_buffer_gdf.to_file(ruta_gpkg_salida, layer="CICLV_ACC_20_23", driver="GPKG")

## Por Ciclovía (bici blanca)

In [19]:
#rutas

ruta_cv = r"C:\Users\UTIC\Desktop\CV_2024\CV_2024.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\4. RED CV\ACCIDENTES\BICI_ BLANCA.gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\4. RED CV\ACCIDENTES\CV_BiciBlanca_20_23.gpkg"

In [14]:
import geopandas as gpd

# Cargar el archivo shapefile de ciclovías
ciclovias = gpd.read_file(ruta_cv)

# Realizar dissolve por el campo CICLV
ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
ciclovias_buffer = ciclovias_dissolved.copy()
ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# Crear un GeoDataFrame a partir del buffer
ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

# Cargar capa geopackage
puntos_aforo = gpd.read_file(ruta_pun_afo, layer="BICI_ BLANCA")

#filtrar capa por año
puntos_aforo_19 = puntos_aforo[puntos_aforo['AÑO'] == 2019]
puntos_aforo_20 = puntos_aforo[puntos_aforo['AÑO'] == 2020]
puntos_aforo_21 = puntos_aforo[puntos_aforo['AÑO'] == 2021]
puntos_aforo_22 = puntos_aforo[puntos_aforo['AÑO'] == 2022]
puntos_aforo_23 = puntos_aforo[puntos_aforo['AÑO'] == 2023]

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    aforo_total = []
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        # total_aforo = aforo_intersect["NUMPOINTS"].count()
        total_puntos = len(aforo_intersect)
        # promedio_aforo = total_aforo / total_puntos if total_puntos > 0 else 0
        
        # aforo_total.append(total_aforo)
        puntos_total.append(total_puntos)
        # ciclovias_buffer_gdf.loc[idx, f"ACC_{year_suffix}"] = total_aforo
        ciclovias_buffer_gdf.loc[idx, f"PUN_{year_suffix}"] = total_puntos
        # ciclovias_buffer_gdf.loc[idx, f"PROM_{year_suffix}"] = promedio_aforo

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_19, "19")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_20, "20")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_21, "21")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo_22, "22")


In [15]:
import numpy as np

# Función para calcular la tasa de cambio de delitos
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de DEL para cada año
        del_19 = row["PUN_19"]
        del_20 = row["PUN_20"]
        del_21 = row["PUN_21"]
        del_22 = row["PUN_22"]
        
        # Obtener el año de construcción de la ciclovía y convertirlo a entero
        ano_const = int(row["ANO_CONST"])
        
        # Filtrar los años con información válida
        values = [(19, del_19), (20, del_20), (21, del_21), (22, del_22)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio basadas en el año de construcción
        tasas_cambio = []
        if ano_const in [2020, 2021, 2022, 2023]:
            # Caso A: Comparar el año previo a la construcción, el año de construcción y el año posterior
            relevant_years = [(ano_const - 1) % 100, ano_const % 100, (ano_const + 1) % 100]
            relevant_values = [v for v in values if v[0] in relevant_years]
            
            print(f"Index: {idx}, Año Construcción: {ano_const}, Relevant Years: {relevant_years}, Relevant Values: {relevant_values}")
            
            if len(relevant_values) < 2:
                ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
                continue
            
            for i in range(len(relevant_values) - 1):
                año_inicial, valor_inicial = relevant_values[i]
                año_final, valor_final = relevant_values[i + 1]
                tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
                tasas_cambio.append(tasa_cambio)
                
        else:
            # Caso B: Comparar todos los años disponibles
            for i in range(len(values) - 1):
                año_inicial, valor_inicial = values[i]
                año_final, valor_final = values[i + 1]
                tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
                tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio) if tasas_cambio else np.nan
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
ciclovias_buffer_gdf = calcular_tasa_cambio(ciclovias_buffer_gdf)

In [18]:
ciclovias_buffer_gdf['TASA_CAMBIO'].max()d

1.0

In [20]:
# Guardar el resultado en un archivo geopackage
ciclovias_buffer_gdf.to_file(ruta_gpkg_salida, layer="CICLV_BICIBLANCA_20_23", driver="GPKG")

## Estaciones MiBici

In [5]:
#rutas

ruta_cv = r"C:\Users\UTIC\Desktop\CV_2024\CV_2024.shp"

ruta_pun_afo = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\Estaciones\ESTACIONES CON VIAJES.gpkg"

ruta_gpkg_salida = r"C:\Users\UTIC\Documents\Alejandro\Docs\INFORME MOV\6. MIBICI\Estaciones\CV_Estaciones_20_23.gpkg"

In [6]:
import geopandas as gpd

# Cargar el archivo shapefile de ciclovías
ciclovias = gpd.read_file(ruta_cv)

# Realizar dissolve por el campo CICLV
ciclovias_dissolved = ciclovias.dissolve(by="CICLV")

# Realizar el buffer de 70 metros sin disolver el resultado y conservar la información original
ciclovias_buffer = ciclovias_dissolved.copy()
ciclovias_buffer['geometry'] = ciclovias_dissolved.buffer(70)

# Crear un GeoDataFrame a partir del buffer
ciclovias_buffer_gdf = gpd.GeoDataFrame(ciclovias_buffer)

# Cargar capa geopackage
puntos_aforo = gpd.read_file(ruta_pun_afo, layer="ESTACIONES CON VIAJES")

# #filtrar capa por año
# puntos_aforo_19 = puntos_aforo[puntos_aforo['AÑO'] == 2019]
# puntos_aforo_20 = puntos_aforo[puntos_aforo['AÑO'] == 2020]
# puntos_aforo_21 = puntos_aforo[puntos_aforo['AÑO'] == 2021]
# puntos_aforo_22 = puntos_aforo[puntos_aforo['AÑO'] == 2022]
# puntos_aforo_23 = puntos_aforo[puntos_aforo['AÑO'] == 2023]

# Función para procesar cada capa de puntos de aforo
def procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, year_suffix):
    aforo_total = []
    puntos_total = []
    
    for idx, buffer_row in ciclovias_buffer_gdf.iterrows():
        buffer_geom = buffer_row.geometry
        aforo_intersect = puntos_aforo[puntos_aforo.intersects(buffer_geom)]
        
        total_aforo = aforo_intersect[f"Viajes_20{year_suffix}"].sum()
        # total_puntos = len(aforo_intersect)
        # promedio_aforo = total_aforo / total_puntos if total_puntos > 0 else 0
        
        aforo_total.append(total_aforo)
        # puntos_total.append(total_puntos)
        ciclovias_buffer_gdf.loc[idx, f"VIAJ_{year_suffix}"] = total_aforo
        # ciclovias_buffer_gdf.loc[idx, f"PUN_{year_suffix}"] = total_puntos
        # ciclovias_buffer_gdf.loc[idx, f"PROM_{year_suffix}"] = promedio_aforo

    return ciclovias_buffer_gdf

# Procesar cada capa de puntos de aforo
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, "20")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, "21")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, "22")
ciclovias_buffer_gdf = procesar_aforo(ciclovias_buffer_gdf, puntos_aforo, "23")

In [7]:
import numpy as np

# Función para calcular la tasa de cambio ciclista
def calcular_tasa_cambio(ciclovias_gdf):
    for idx, row in ciclovias_gdf.iterrows():
        # Obtener los valores de PROM para cada año
        prom_20 = row["VIAJ_20"]
        prom_21 = row["VIAJ_21"]
        prom_22 = row["VIAJ_22"]
        prom_23 = row["VIAJ_23"]
        
        # Filtrar los años con información válida
        values = [(20, prom_20), (21, prom_21), (22, prom_22), (23, prom_23)]
        values = [v for v in values if v[1] != 0]
        
        if len(values) < 2:
            # Si no hay suficientes años con datos, establecer la tasa de cambio como NaN
            ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = np.nan
            continue
        
        # Calcular las tasas de cambio entre años consecutivos
        tasas_cambio = []
        for i in range(len(values) - 1):
            año_inicial, valor_inicial = values[i]
            año_final, valor_final = values[i + 1]
            tasa_cambio = ((valor_final - valor_inicial) / valor_inicial)
            tasas_cambio.append(tasa_cambio)
        
        # Calcular el promedio de las tasas de cambio
        tasa_cambio_promedio = np.mean(tasas_cambio)
        ciclovias_gdf.loc[idx, "TASA_CAMBIO"] = tasa_cambio_promedio

    return ciclovias_gdf

# Aplicar la función de cálculo de tasa de cambio
ciclovias_buffer_gdf = calcular_tasa_cambio(ciclovias_buffer_gdf)


In [9]:
# Guardar el resultado en un archivo geopackage
ciclovias_buffer_gdf.to_file(ruta_gpkg_salida, layer="CICLV_VisjMiBici_20_23", driver="GPKG")